In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import gridspec 
import random
import xgboost as xgb
from sklearn import preprocessing
from math import sqrt
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
top = np.percentile(train.Purchase,99.5)
train = train[ (train['Purchase']<top) ]
#test1.head()
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)
test['Product_Category_3']=test['Product_Category_3'].astype(int)
train['Stay_In_Current_City_Years'].replace('4+',4,inplace=True)
test['Stay_In_Current_City_Years'].replace('4+',4,inplace=True)

new_age = {'0-17':0,'18-25':1,'26-35':2,'36-45':3,'46-50':4,'51-55':5,'55+':6}
train.Age.replace(new_age,inplace=True)
test.Age.replace(new_age,inplace=True)

new_city = {'A':0, 'B':1, 'C':2}
train.City_Category.replace(new_city,inplace=True)
test.City_Category.replace(new_city,inplace=True)

new_gender = {'F':0, 'M':1}
train.Gender.replace(new_gender,inplace=True)
test.Gender.replace(new_gender,inplace=True)

user_id_to_category_map = {}
customer_purchase_power = train.groupby("User_ID")["Purchase"].sum()
customer_purchase_power.median()

513874.0

In [20]:
values = customer_purchase_power.iteritems()
for key, val in values:
    if val <= 143662.0:
        user_id_to_category_map[key] = 1
    elif val <= 202156.0:
        user_id_to_category_map[key] = 2
    elif val <= 275408.0:
        user_id_to_category_map[key] = 3
    elif val <= 378451.0:
        user_id_to_category_map[key] = 4
    elif val <= 513874.0:
        user_id_to_category_map[key] = 5
    elif val <= 687241.0:
        user_id_to_category_map[key] = 6
    elif val <= 934097.0:
        user_id_to_category_map[key] = 7
    elif val <= 1332584.0:
        user_id_to_category_map[key] = 8
    elif val <= 2029891.0:
        user_id_to_category_map[key] = 9
    else:
        user_id_to_category_map[key] = 10
customer_purchase_power.median()

513874.0

In [21]:
def get_customer_category(user_id):
    if user_id in user_id_to_category_map:
        return user_id_to_category_map[user_id]
    return 5
train["buying_power"] = list(map(lambda user_id: get_customer_category(user_id), train["User_ID"]))
test["buying_power"] = list(map(lambda user_id: get_customer_category(user_id), test["User_ID"]))

In [22]:
train["User_ID_MinPrice"] = train.groupby(['User_ID'])['Purchase'].transform('min')
userID_min_dict = train.groupby(['User_ID'])['Purchase'].min().to_dict()
test['User_ID_MinPrice'] = test['User_ID'].apply(lambda x:userID_min_dict.get(x,0))

train["User_ID_MaxPrice"] = train.groupby(['User_ID'])['Purchase'].transform('max')
userID_max_dict = train.groupby(['User_ID'])['Purchase'].max().to_dict()
test['User_ID_MaxPrice'] = test['User_ID'].apply(lambda x:userID_max_dict.get(x,0))

train["Product_ID_MinPrice"] = train.groupby(['Product_ID'])['Purchase'].transform('min')
productID_min_dict = train.groupby(['Product_ID'])['Purchase'].min().to_dict()
test['Product_ID_MinPrice'] = test['Product_ID'].apply(lambda x:productID_min_dict.get(x,0))

train["Product_ID_MaxPrice"] = train.groupby(['Product_ID'])['Purchase'].transform('max')
productID_max_dict = train.groupby(['Product_ID'])['Purchase'].max().to_dict()
test['Product_ID_MaxPrice'] = test['Product_ID'].apply(lambda x:productID_max_dict.get(x,0))

train["User_ID_Count"] = train.groupby(['User_ID'])['User_ID'].transform('count')
uID_count = train.groupby(['User_ID']).size().to_dict()
test['User_ID_Count'] = test['User_ID'].apply(lambda x:uID_count.get(x,0))

train["Product_ID_Count"] = train.groupby(['Product_ID'])['Product_ID'].transform('count')
pID_count = train.groupby(['Product_ID']).size().to_dict()
test['Product_ID_Count'] = test['Product_ID'].apply(lambda x:pID_count.get(x,0))

train["User_ID_MeanPrice"] = train.groupby(['User_ID'])['Purchase'].transform('mean')
userID_mean_dict = train.groupby(['User_ID'])['Purchase'].mean().to_dict()
test['User_ID_MeanPrice'] = test['User_ID'].apply(lambda x:userID_mean_dict.get(x,0))

train["Product_ID_MeanPrice"] = train.groupby(['Product_ID'])['Purchase'].transform('mean')
productID_mean_dict = train.groupby(['Product_ID'])['Purchase'].mean().to_dict()
test['Product_ID_MeanPrice'] = test['Product_ID'].apply(lambda x:productID_mean_dict.get(x,0))

train["Product_Cat1_MeanPrice"] = train.groupby(['Product_Category_1'])['Purchase'].transform('mean')
pc1_mean_dict = train.groupby(['Product_Category_1'])['Purchase'].mean().to_dict()
test['Product_Cat1_MeanPrice'] = test['Product_Category_1'].apply(lambda x:pc1_mean_dict.get(x,0))

train["Product_Cat1_MinPrice"] = train.groupby(['Product_Category_1'])['Purchase'].transform('min')
pc1_min_dict = train.groupby(['Product_Category_1'])['Purchase'].min().to_dict()
test['Product_Cat1_MinPrice'] = test['Product_Category_1'].apply(lambda x:pc1_min_dict.get(x,0))

train["Product_Cat1_MaxPrice"] = train.groupby(['Product_Category_1'])['Purchase'].transform('max')
pc1_max_dict = train.groupby(['Product_Category_1'])['Purchase'].max().to_dict()
test['Product_Cat1_MaxPrice'] = test['Product_Category_1'].apply(lambda x:pc1_max_dict.get(x,0))

train["Product_Cat2_MeanPrice"] = train.groupby(['Product_Category_2'])['Purchase'].transform('mean')
pc2_mean_dict = train.groupby(['Product_Category_2'])['Purchase'].mean().to_dict()
test['Product_Cat2_MeanPrice'] = test['Product_Category_2'].apply(lambda x:pc2_mean_dict.get(x,0))

train["Product_Cat2_MinPrice"] = train.groupby(['Product_Category_2'])['Purchase'].transform('min')
pc2_min_dict = train.groupby(['Product_Category_2'])['Purchase'].min().to_dict()
test['Product_Cat2_MinPrice'] = test['Product_Category_2'].apply(lambda x:pc2_min_dict.get(x,0))

train["Product_Cat2_MaxPrice"] = train.groupby(['Product_Category_2'])['Purchase'].transform('max')
pc2_max_dict = train.groupby(['Product_Category_2'])['Purchase'].max().to_dict()
test['Product_Cat2_MaxPrice'] = test['Product_Category_2'].apply(lambda x:pc2_max_dict.get(x,0))

train["Product_Cat3_MeanPrice"] = train.groupby(['Product_Category_3'])['Purchase'].transform('mean')
pc3_mean_dict = train.groupby(['Product_Category_3'])['Purchase'].mean().to_dict()
test['Product_Cat3_MeanPrice'] = test['Product_Category_3'].apply(lambda x:pc3_mean_dict.get(x,0))

train["Product_Cat3_MinPrice"] = train.groupby(['Product_Category_3'])['Purchase'].transform('min')
pc3_min_dict = train.groupby(['Product_Category_3'])['Purchase'].min().to_dict()
test['Product_Cat3_MinPrice'] = test['Product_Category_3'].apply(lambda x:pc3_min_dict.get(x,0))

train["Product_Cat3_MaxPrice"] = train.groupby(['Product_Category_3'])['Purchase'].transform('max')
pc3_max_dict = train.groupby(['Product_Category_3'])['Purchase'].max().to_dict()
test['Product_Cat3_MaxPrice'] = test['Product_Category_3'].apply(lambda x:pc3_max_dict.get(x,0))

userID_25p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['User_ID_25PercPrice'] = train['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))
test['User_ID_25PercPrice'] = test['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))

userID_50p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['User_ID_50PercPrice'] = train['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))
test['User_ID_50PercPrice'] = test['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))

userID_75p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['User_ID_75PercPrice'] = train['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))
test['User_ID_75PercPrice'] = test['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))

productID_25p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Product_ID_25PercPrice'] = train['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))
test['Product_ID_25PercPrice'] = test['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))

productID_50p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Product_ID_50PercPrice'] = train['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))
test['Product_ID_50PercPrice'] = test['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))

productID_75p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Product_ID_75PercPrice'] = train['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))
test['Product_ID_75PercPrice'] = test['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))

pc1_25p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Product_Cat1_25PercPrice'] = train['Product_Category_1'].apply(lambda x:pc1_25p_dict.get(x,0))
test['Product_Cat1_25PercPrice'] = test['Product_Category_1'].apply(lambda x:pc1_25p_dict.get(x,0))

pc1_50p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Product_Cat1_50PercPrice'] = train['Product_Category_1'].apply(lambda x:pc1_50p_dict.get(x,0))
test['Product_Cat1_50PercPrice'] = test['Product_Category_1'].apply(lambda x:pc1_50p_dict.get(x,0))

pc1_75p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Product_Cat1_75PercPrice'] = train['Product_Category_1'].apply(lambda x:pc1_75p_dict.get(x,0))
test['Product_Cat1_75PercPrice'] = test['Product_Category_1'].apply(lambda x:pc1_75p_dict.get(x,0))


pc2_25p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Product_Cat2_25PercPrice'] = train['Product_Category_2'].apply(lambda x:pc2_25p_dict.get(x,0))
test['Product_Cat2_25PercPrice'] = test['Product_Category_2'].apply(lambda x:pc2_25p_dict.get(x,0))

pc2_50p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Product_Cat2_50PercPrice'] = train['Product_Category_2'].apply(lambda x:pc2_50p_dict.get(x,0))
test['Product_Cat2_50PercPrice'] = test['Product_Category_2'].apply(lambda x:pc2_50p_dict.get(x,0))

pc2_75p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Product_Cat2_75PercPrice'] = train['Product_Category_2'].apply(lambda x:pc2_75p_dict.get(x,0))
test['Product_Cat2_75PercPrice'] = test['Product_Category_2'].apply(lambda x:pc2_75p_dict.get(x,0))

pc3_25p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Product_Cat3_25PercPrice'] = train['Product_Category_3'].apply(lambda x:pc3_25p_dict.get(x,0))
test['Product_Cat3_25PercPrice'] = test['Product_Category_3'].apply(lambda x:pc3_25p_dict.get(x,0))

pc3_50p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Product_Cat3_50PercPrice'] = train['Product_Category_3'].apply(lambda x:pc3_50p_dict.get(x,0))
test['Product_Cat3_50PercPrice'] = test['Product_Category_3'].apply(lambda x:pc3_50p_dict.get(x,0))

pc3_75p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Product_Cat3_75PercPrice'] = train['Product_Category_3'].apply(lambda x:pc3_75p_dict.get(x,0))
test['Product_Cat3_75PercPrice'] = test['Product_Category_3'].apply(lambda x:pc3_75p_dict.get(x,0))

train["Occupation_Count"] = train.groupby(['Occupation'])['Occupation'].transform('count')
occupation_count_dict = train.groupby(['Occupation']).size().to_dict()
test['Occupation_Count'] = test['Occupation'].apply(lambda x:occupation_count_dict.get(x,0))

train["Product_Category_1_Count"] = train.groupby(['Product_Category_1'])['Product_Category_1'].transform('count')
pc1_count_dict = train.groupby(['Product_Category_1']).size().to_dict()
test['Product_Category_1_Count'] = test['Product_Category_1'].apply(lambda x:pc1_count_dict.get(x,0))

train["Product_Category_2_Count"] = train.groupby(['Product_Category_2'])['Product_Category_2'].transform('count')
pc2_count_dict = train.groupby(['Product_Category_2']).size().to_dict()
test['Product_Category_2_Count'] = test['Product_Category_2'].apply(lambda x:pc2_count_dict.get(x,0))

train["Product_Category_3_Count"] = train.groupby(['Product_Category_3'])['Product_Category_3'].transform('count')
pc3_count_dict = train.groupby(['Product_Category_3']).size().to_dict()
test['Product_Category_3_Count'] = test['Product_Category_3'].apply(lambda x:pc3_count_dict.get(x,0))

In [23]:
auto3 = pd.DataFrame()
auto3['User_ID'] = test['User_ID']
auto3['Product_ID'] = test['Product_ID']
cat_col=["User_ID", "Product_ID"]
for var in cat_col:
    lb = preprocessing.LabelEncoder()
    full_var_data = pd.concat((train[var], test[var]), axis=0).astype('str')
    lb.fit(full_var_data)
    train[var] = lb.transform(train[var].astype('str'))
    test[var] = lb.transform(test[var].astype('str'))

In [24]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\tsans\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\tsans\AppData\Local\Temp\tmp0srf2ycw
  JVM stdout: C:\Users\tsans\AppData\Local\Temp\tmp0srf2ycw\h2o_tsans_started_from_python.out
  JVM stderr: C:\Users\tsans\AppData\Local\Temp\tmp0srf2ycw\h2o_tsans_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_tsans_nvkm4a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.111 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [25]:
train['Stay_In_Current_City_Years'] = train['Stay_In_Current_City_Years'].astype(int)
test['Stay_In_Current_City_Years'] = test['Stay_In_Current_City_Years'].astype(int)

In [26]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)
print('done')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
done


In [27]:
x = train.columns
y = "Purchase"
x.remove(y)

In [28]:
aml = H2OAutoML(max_models=12, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
14:34:16.980: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [29]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201228_143416,5.70948e+06,2389.45,5.70948e+06,1745.77,nan
StackedEnsemble_BestOfFamily_AutoML_20201228_143416,5.74758e+06,2397.41,5.74758e+06,1751.79,nan
GBM_5_AutoML_20201228_143416,5.75964e+06,2399.93,5.75964e+06,1753.37,nan
GBM_4_AutoML_20201228_143416,5.76707e+06,2401.47,5.76707e+06,1756.52,nan
GBM_grid__1_AutoML_20201228_143416_model_2,5.79899e+06,2408.11,5.79899e+06,1756.79,nan
GBM_3_AutoML_20201228_143416,5.8116e+06,2410.73,5.8116e+06,1765.6,nan
GBM_2_AutoML_20201228_143416,5.83748e+06,2416.09,5.83748e+06,1771.08,nan
GBM_1_AutoML_20201228_143416,5.86851e+06,2422.5,5.86851e+06,1776.57,nan
GBM_grid__1_AutoML_20201228_143416_model_1,5.88586e+06,2426.08,5.88586e+06,1779.41,nan
DRF_1_AutoML_20201228_143416,5.89499e+06,2427.96,5.89499e+06,1775.21,0.314585


In [30]:
predleader = aml.leader.predict(test)
pred = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [32]:
dfpred = h2o.as_list(predleader)
auto3['Purchase'] = dfpred
#sum(n < 0 for n in auto3["Purchase"].values.flatten())
auto3[auto3["Purchase"] < 0] = 0
auto3.to_csv('auto3.csv',index=False)

In [34]:
h2o.shutdown()

<ipython-input-34-1edf85295eae>:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
